---
format: 
  html:
    toc: true
    code-fold: true
execute:
  echo: true
  eval: false
---

# Data Sources

In this section, I will show how to get the data for this project. I will give an example of using API to call the **foot traffic data** provided by [Dewey](https://www.deweydata.io/). There is also an official guide on youtube that you can refer to [here](https://www.youtube.com/watch?v=SlJAZ_91UPg). For the **spend pattern data**, the process would be exactly the same except for the `API Key` and `file path` obtained from the website. Here are the details about the data sets:

- [ADVAN monthly foot traffic data](https://app.deweydata.io/products/5acc9f39-1ca6-4535-b3ff-38f6b9baf85e/package). It includes aggregated raw counts of visits to POIs from a panel of mobile devices over a given month, detailing how often people visit, how long they stay, where they came from, where else they go, and more.
- [SafeGraph monthly spend patterns data](https://app.deweydata.io/products/eb6e748a-0fdd-4bc7-9dd7-bbed0890948d/package). It includes aggregated, anonymized credit and debit transaction data associated to specific stores, including median spend per day, median spend per customer, and other detailed statistics, as well as where else consumers spend money and the breakdown of online/offline spending.

The two data sets can be joined by a shared column called PlaceKey, which is the unique and persistent ID tied to a POI.

First, we need to install the `deweydatapy` package, and then import it to use the API.

In [ ]:
pip install deweydatapy

In [1]:
import deweydatapy as ddp
import pandas as pd
import os
import geopandas as gpd

Then, we need to get the `API key` and the `file path` from the Dewey website. After that, we can use the `get_meta` function to have an overview of the wanted data. It suggests that the total size of the data is about 750,000 MB, and is aggregated by month from 1/1/2019 to 10/1/2024.

In [2]:
# Monthly patterns api key
apikey_ = "Your API Key"

# File path
path = "Your file path"

In [3]:
meta = ddp.get_meta(apikey_, path, print_meta=True)

 
Metadata summary ------------------------------------------------
Total number of files: 3,607
Total files size (MB): 748,164.8
Date aggregation: MONTH
Date partition column: DATE_RANGE_START
Data min available date: 2019-01-01
Data max available date: 2024-10-01
-----------------------------------------------------------------
 


Then, we can specify the date range for which we want to get the data. If we did not specify a specific date range, the API will return the whole data of a year and the file size would be too large for the computer to handle. So, we will use the `start_date` and `end_date` parameters to get the data of July from 2019 to 2024. The date format should be `YYYY-MM-DD`. For example, to get the data for the month of July 2021, we can set the `start_date` to `2021-07-01` and the `end_date` to `2021-07-31`. 

Here we can write a for loop to get the data of each year and store them in a list, the data size is 62.5 GB in total.


In [4]:
year_var = [str(year) for year in range(19, 25)]
file_lists = {}

for i in year_var:
    start_date = "20" + i + "-07-01"
    end_date = "20" + i + "-07-31"

    file_list = ddp.get_file_list(apikey_, path, start_date=start_date, end_date=end_date, print_info=True)

    dataframe_name = "file_df_" + i
    file_lists[dataframe_name] = pd.DataFrame(file_list)

Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 1
Total number of files: 60
Total files size (MB): 12,389.41
Average single file size (MB): 206.49
Date partition column: DATE_RANGE_START
Expires at: 2024-11-15T13:58:41.429Z
-----------------------------------------------------------------
Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 1
Total number of files: 47
Total files size (MB): 9,768.66
Average single file size (MB): 207.84
Date partition column: DATE_RANGE_START
Expires at: 2024-11-15T13:58:42.285Z
-----------------------------------------------------------------
Files information collection completed.
 
Files information summary ---------------------------------------
Total number of pages: 1
Total number of files: 48
Total files size (MB): 10,033.68
Average single file size (MB): 209.03
Date partition column: DA

In [5]:
size = []

for i in year_var:
    file_df = "file_df_" + i
    size.append(file_lists[file_df]["file_size_bytes"].sum())

sum(size) / (1024 ** 3)

62.5066275279969

Finally, we can use the `download_files` function to download the data to the specified file path, and use the `CITY` and `REGION` parameters to filter the data to Philadelphia. The data will be saved in a `.csv` file format.

In [ ]:
for i in year_var:
    save_path = "data/ADVAN/data_" + i
    file_df = "file_df_" + i
    df_to_download = file_lists[file_df]

    ddp.download_files(df_to_download, save_path, skip_exists=True)

In [ ]:
# 2019
folder_path = "data/ADVAN/data_19/"
files = [file for file in os.listdir(folder_path)]
dfs = []

for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

df_final_19 = pd.concat(dfs, ignore_index=True)

df_philly_19 = df_final_19.loc[(df_final_19["CITY"] == "Philadelphia") & (df_final_19["REGION"] == "PA")]

df_final_19.to_csv("data/ADVAN/df_19.csv", index=False)
df_philly_19.to_csv("data/ADVAN/df_philly_19.csv", index=False)

#2020
folder_path = "data/ADVAN/data_20/"
files = [file for file in os.listdir(folder_path)]
dfs = []

for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

df_final_20 = pd.concat(dfs, ignore_index=True)

df_philly_20 = df_final_20.loc[(df_final_20["CITY"] == "Philadelphia") & (df_final_20["REGION"] == "PA")]

df_final_20.to_csv("data/ADVAN/df_20.csv", index=False)
df_philly_20.to_csv("data/ADVAN/df_philly_20.csv", index=False)

#2021
folder_path = "data/ADVAN/data_21/"
files = [file for file in os.listdir(folder_path)]
dfs = []

for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

df_final_21 = pd.concat(dfs, ignore_index=True)

df_philly_21 = df_final_21.loc[(df_final_21["CITY"] == "Philadelphia") & (df_final_21["REGION"] == "PA")]

df_final_21.to_csv("data/ADVAN/df_21.csv", index=False)
df_philly_21.to_csv("data/ADVAN/df_philly_21.csv", index=False)

#2022
folder_path = "data/ADVAN/data_22/"
files = [file for file in os.listdir(folder_path)]
dfs = []

for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

df_final_22 = pd.concat(dfs, ignore_index=True)

df_philly_22 = df_final_22.loc[(df_final_22["CITY"] == "Philadelphia") & (df_final_22["REGION"] == "PA")]

df_final_22.to_csv("data/ADVAN/df_22.csv", index=False)
df_philly_22.to_csv("data/ADVAN/df_philly_22.csv", index=False)

#2023
folder_path = "data/ADVAN/data_23/"
files = [file for file in os.listdir(folder_path)]
dfs = []

for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

df_final_23 = pd.concat(dfs, ignore_index=True)

df_philly_23 = df_final_23.loc[(df_final_23["CITY"] == "Philadelphia") & (df_final_23["REGION"] == "PA")]

df_final_23.to_csv("data/ADVAN/df_23.csv", index=False)
df_philly_23.to_csv("data/ADVAN/df_philly_23.csv", index=False)

#2024
folder_path = "data/ADVAN/data_24/"
files = [file for file in os.listdir(folder_path)]
dfs = []

for file in files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

df_final_24 = pd.concat(dfs, ignore_index=True)

df_philly_24 = df_final_24.loc[(df_final_24["CITY"] == "Philadelphia") & (df_final_24["REGION"] == "PA")]

df_final_24.to_csv("data/ADVAN/df_24.csv", index=False)
df_philly_24.to_csv("data/ADVAN/df_philly_24.csv", index=False)

Now, we have downloaded the foot traffic data and spend pattern data. For the sake of computer RAM capacity, we can restart the kernel and load the data in the following sections, where we will filter the data by `NAICS_CODE` and join the two data sets by the `PlaceKey` column.

In [ ]:
# Load the ADVAN data
advan_raw = pd.read_csv('data/ADVAN/df_philly_19_24.csv')

# Load the SafeGraph data
sg_raw = pd.read_csv('data/SafeGraph/df_philly_19_24.csv')

C:\Users\chenf\AppData\Local\Temp\ipykernel_1524\2411687974.py:2: DtypeWarning: Columns (17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  advan_raw = pd.read_csv('data/ADVAN/df_philly_19_24.csv')


In [ ]:
print(f"Advan foot traffic data has {advan_raw.shape[0]} rows and SafeGraph spend pattern data has {sg_raw.shape[0]} rows.")

Advan foot traffic data has 373583 rows and SafeGraph spend pattern data has 24083 rows.


In [ ]:
advan_raw['NAICS_CODE'] = advan_raw['NAICS_CODE'].astype(str)

advan = advan_raw.loc[
    ~advan_raw['POPULARITY_BY_HOUR'].isnull()
    ].loc[
    advan_raw['NAICS_CODE'].str.startswith('42') |  # Wholesale Trade
    advan_raw['NAICS_CODE'].str.startswith('44') |  # Retail Trade
    advan_raw['NAICS_CODE'].str.startswith('45') |  # Retail Trade
    advan_raw['NAICS_CODE'].str.startswith('72')    # Accommodation and Food Services
]

advan_gdf = gpd.GeoDataFrame(
    advan, 
    geometry=gpd.points_from_xy(advan['LONGITUDE'], advan['LATITUDE']),
    crs='EPSG:4326'
)

Finally, we can save the joined data of each year to `.geojson` file for further analysis.

In [ ]:
advan_sg = advan_gdf.merge(
    sg_raw,
    left_on=['PLACEKEY', 'DATE_RANGE_START'],
    right_on=['PLACEKEY', 'SPEND_DATE_RANGE_START'],
    how='left'
)

# Convert to the same CRS
advan_gdf = advan_gdf.to_crs(2272)
advan_sg = advan_sg.to_crs(2272)

# Select columns
advan_sg = advan_sg[['PLACEKEY', 'LOCATION_NAME', 'TOP_CATEGORY', 'LATITUDE', 'LONGITUDE', 'DATE_RANGE_START', 'RAW_VISIT_COUNTS', 'RAW_VISITOR_COUNTS', 'RAW_TOTAL_SPEND', 'RAW_NUM_TRANSACTIONS', 'RAW_NUM_CUSTOMERS']]
advan_sg['DATE_RANGE_START'] = advan_sg['DATE_RANGE_START'].dt.year

advan_sg = gpd.GeoDataFrame(advan_sg, geometry=gpd.points_from_xy(advan_sg['LONGITUDE'], advan_sg['LATITUDE']), crs='EPSG:4326')

# Save the data
advan_sg.to_file('data/advan_sg.geojson', driver='GeoJSON')